In [1]:
import os
import csv
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import AllChem
from networkx.algorithms import isomorphism
import networkx as nx

lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

def read_sdf(file_path):
    mol = Chem.SDMolSupplier(file_path)[0]
    return mol

def mol_to_graph(mol):
    G = nx.Graph()
    
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atom_type=atom.GetSymbol())
    
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
    
    return G

def calculate_rmsd(mol1, mol2, atom_map):
    coords1 = mol1.GetConformer().GetPositions()
    coords2 = mol2.GetConformer().GetPositions()
    
    mapped_coords1 = np.array([coords1[i] for i in atom_map.keys()])
    mapped_coords2 = np.array([coords2[i] for i in atom_map.values()])
    
    diff = mapped_coords1 - mapped_coords2
    return np.sqrt(np.mean(np.sum(diff**2, axis=1)))

def calculate_rmsd_without_alignment(sdf_file1, sdf_file2):
    mol1 = read_sdf(sdf_file1)
    mol2 = read_sdf(sdf_file2)
    
    graph1 = mol_to_graph(mol1)
    if mol2 is None:
        return None
    else:
        graph2 = mol_to_graph(mol2)
    
    nm = isomorphism.GraphMatcher(graph1, graph2, 
                                  node_match=lambda n1, n2: n1['atom_type'] == n2['atom_type'])
    
    if nm.is_isomorphic():
        atom_map = nm.mapping
        rmsd = calculate_rmsd(mol1, mol2, atom_map)
        return rmsd
    else:
        return None

In [2]:
def process_sdfs(inference_dir,reference_dir,output_csv):
    results = []
    
    for filename in os.listdir(inference_dir):
        if filename.endswith(".sdf"):
            prot = filename.split('.')[0]
            pred_sdf = os.path.join(inference_dir, filename)
            ref_sdf = os.path.join(reference_dir, prot, f"{prot}_ligand.sdf")
            
            if not os.path.exists(ref_sdf):
                print(f"Warning: Reference file not found for {prot}")
                results.append((prot, -1))
                continue
            
            rmsd = calculate_rmsd_without_alignment(ref_sdf, pred_sdf)
            if rmsd is None:
                results.append((prot, -1))
            else:
                results.append((prot, rmsd))
    
    # Write results to CSV
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pdb_id', 'rmsd'])
        for prot, rmsd in results:
            writer.writerow([prot, f"{rmsd:.4f}" if rmsd != -1 else "-1"])

inference_dir = "./posebusters_benchmark/docking_results"
reference_dir = "/mnt/data/posebusters/posebusters_benchmark_set"
output_csv = "unaligned_rmsd_results.csv"

process_sdfs(inference_dir, reference_dir, output_csv)
print(f"Results have been written to {output_csv}")

Results have been written to unaligned_rmsd_results.csv


In [3]:
file_path = './unaligned_rmsd_results.csv'
data = pd.read_csv(file_path)

# Calculate Success Rate
valid_rmsd = data[data['rmsd'] != -1]
rmsd_less_than_2 = valid_rmsd[valid_rmsd['rmsd'] < 2]
rmsd_less_than_2_ratio = len(rmsd_less_than_2) / len(data)

print(f"RMSD < 2: {rmsd_less_than_2_ratio:.2%}")

RMSD < 2: 10.05%
